Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
#DAF understanding
#tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
#tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
#weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, H])) H=number of nodes of the hidden layer
#biases1 = tf.Variable(tf.zeros([H]))
#logits1 = tf.matmul(tf_train_dataset, weights1) + biases1

#DAF: understanding (with the help of numpy)
#my_features = np.array([[1., 2.], [3., 4.], [5., 6.], [7., 8.]])
#my_weights = np.array([0.4, 0.5])
#my_predictions = np.dot(my_features, my_weights) # note that the weights are on the right
#print(my_predictions) # which has 4 elements since my_features has 4 rows
#train_data_ = np.ndarray(shape=(16,28*28), dtype=np.float32)
#train_data_ = np.random.rand(16,28*28)

train_data_ = np.full((16, 28*28), 1.0)
train_labels_ = np.full((16, 10), 1.0)
weights1_ = np.random.rand(28*28, 1024)
biases1_ = np.full(1024,1.0)
print("DATASET: ", train_data_.shape , train_labels_.shape)
print("PARAMS: ", weights1_.shape, biases1_.shape)
prod_ = np.dot(train_data_, weights1_)
logits1_ = prod_ + biases1_
print("LOGITS: ", prod_.shape, logits1_.shape)

DATASET:  (16, 784) (16, 10)
PARAMS:  (784, 1024) (1024,)
LOGITS:  (16, 1024) (16, 1024)


In [6]:
#import time
#start_time = time.time()
#train_dataset_orig = train_dataset
#train_labels_orig = train_labels
#train_dataset = train_dataset[0:32]
#train_labels = train_labels[0:32]

In [7]:
#image_size = 28
#num_labels = 10
#num_channels = 1 # grayscale

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels)) #shape=(16,28,28,1)
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #shape=(16,10)

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)) #shape=(5,5,1,16)
  layer1_biases = tf.Variable(tf.zeros([depth])) #shape=(16,)

  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1)) #shape=(5,5,16,16)
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) #shape=(16,)

  
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1)) #shape=(784,64) , 28//4*28//4*16 = 7*7*16 = 784 
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #shape=(64,)
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1)) #shape(64,10)
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels])) #shape(10,)
  
  # Model.
  def model(data, verbose=False):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    
    if verbose == True:
        print("-> C1.SHAPES WITH STRIDES={}: [input(=data):{} | filter(=layer1_weights):{} -> output(=conv1):{}]".format(
            '[1, 2, 2, 1]', data.shape, layer1_weights.shape, conv1.shape))
        print("-> C1.HIDDEN SHAPES: [conv1:{} | layer1_biases:{} -> output(=hidden1):{}\n".format(
                conv1.shape, layer1_biases.shape, hidden1.shape))
    
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)

    if verbose == True:
        print("--> C2.SHAPES WITH STRIDES={}: [input(=hidden1):{} | filter(=layer2_weights):{} -> output(=conv2):{}]".format(
            '[1, 2, 2, 1]', hidden1.shape, layer2_weights.shape, conv2.shape))
        print("--> C2.HIDDEN SHAPES: [conv2:{} | layer2_biases:{} -> output(=hidden2):{}\n".format(
                conv2.shape, layer2_biases.shape, hidden2.shape))

    shape = hidden2.get_shape().as_list()
    
    reshape2 = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
    #hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    prod3 = tf.matmul(reshape2, layer3_weights)
    hidden3 = tf.nn.relu(prod3 + layer3_biases)
    
    if verbose == True:
        print("---> FULLCONN FIRST LAYER SHAPES MATMUL: [hidden reshaped(=reshape2):{} | layer3_weights:{} -> output(=prod3):{}".format(
                reshape2.shape, layer3_weights.shape, prod3.shape))
        print("---> FULLCONN FIRST LAYER HIDDEN: prod3:{} | layer3_biases:{} -> output(=hidden3):{}\n".format(
            prod3.shape, layer3_biases.shape, hidden3.shape))
    
    #return tf.matmul(hidden, layer4_weights) + layer4_biases
    prod4 = tf.matmul(hidden3, layer4_weights)
    output = prod4 + layer4_biases

    if verbose == True:
        print("----> FULLCONN LAST LAYER SHAPES MATMUL: [hidden3:{} | layer4_weights:{} -> output(=prod4):{}".format(
                hidden3.shape, layer4_weights.shape, prod4.shape))
        print("----> FULLCONN LAST LAYER HIDDEN: prod4:{} | layer4_biases:{} -> output:{}\n".format(
            prod4.shape, layer4_biases.shape, output.shape))    
    
    return output

  # Training computation.
  logits = model(tf_train_dataset, verbose=True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

-> C1.SHAPES WITH STRIDES=[1, 2, 2, 1]: [input(=data):(16, 28, 28, 1) | filter(=layer1_weights):(5, 5, 1, 16) -> output(=conv1):(16, 14, 14, 16)]
-> C1.HIDDEN SHAPES: [conv1:(16, 14, 14, 16) | layer1_biases:(16,) -> output(=hidden1):(16, 14, 14, 16)

--> C2.SHAPES WITH STRIDES=[1, 2, 2, 1]: [input(=hidden1):(16, 14, 14, 16) | filter(=layer2_weights):(5, 5, 16, 16) -> output(=conv2):(16, 7, 7, 16)]
--> C2.HIDDEN SHAPES: [conv2:(16, 7, 7, 16) | layer2_biases:(16,) -> output(=hidden2):(16, 7, 7, 16)

---> FULLCONN FIRST LAYER SHAPES MATMUL: [hidden reshaped(=reshape2):(16, 784) | layer3_weights:(784, 64) -> output(=prod3):(16, 64)
---> FULLCONN FIRST LAYER HIDDEN: prod3:(16, 64) | layer3_biases:(64,) -> output(=hidden3):(16, 64)

----> FULLCONN LAST LAYER SHAPES MATMUL: [hidden3:(16, 64) | layer4_weights:(64, 10) -> output(=prod4):(16, 10)
----> FULLCONN LAST LAYER HIDDEN: prod4:(16, 10) | layer4_biases:(10,) -> output:(16, 10)



In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:

  tf.global_variables_initializer().run()
  print('Initialized')
  
  for step in range(num_steps):
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized


In [9]:
#DAF understanding: offset + repeated batches (i.e. few *different* batches) limiting train_dataset_size=300

batch_size=16
train_dataset_size=300
num_steps = 1001
for step in range(num_steps):

    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (train_dataset_size - batch_size) # para este caso es (step * batch_size) % 284
    # Generate a minibatch.
    print("step {}: batch_data = train_dataset[{}:{}, :, :, :]".format(step, offset, offset + batch_size))
    if(step == 50): break

step 0: batch_data = train_dataset[0:16, :, :, :]
step 1: batch_data = train_dataset[16:32, :, :, :]
step 2: batch_data = train_dataset[32:48, :, :, :]
step 3: batch_data = train_dataset[48:64, :, :, :]
step 4: batch_data = train_dataset[64:80, :, :, :]
step 5: batch_data = train_dataset[80:96, :, :, :]
step 6: batch_data = train_dataset[96:112, :, :, :]
step 7: batch_data = train_dataset[112:128, :, :, :]
step 8: batch_data = train_dataset[128:144, :, :, :]
step 9: batch_data = train_dataset[144:160, :, :, :]
step 10: batch_data = train_dataset[160:176, :, :, :]
step 11: batch_data = train_dataset[176:192, :, :, :]
step 12: batch_data = train_dataset[192:208, :, :, :]
step 13: batch_data = train_dataset[208:224, :, :, :]
step 14: batch_data = train_dataset[224:240, :, :, :]
step 15: batch_data = train_dataset[240:256, :, :, :]
step 16: batch_data = train_dataset[256:272, :, :, :]
step 17: batch_data = train_dataset[272:288, :, :, :]
step 18: batch_data = train_dataset[4:20, :, :, :]
s

In [ ]:
#train_dataset = train_dataset_orig
#train_labels = train_labels_orig
#print("--- %s seconds ---" % (time.time() - start_time))

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [19]:
#image_size = 28
#num_labels = 10
#num_channels = 1 # grayscale

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels)) #shape=(16,28,28,1)
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #shape=(16,10)

  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)) #shape=(5,5,1,16)
  layer1_biases = tf.Variable(tf.zeros([depth])) #shape=(16,)

  
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1)) #shape=(5,5,16,16)
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth])) #shape=(16,)

  
  layer3_weights = tf.Variable(tf.truncated_normal(
      [28 // 4 * 28 // 4 * depth, num_hidden], stddev=0.1)) #shape=(784,64) , 28//4*28//4*16 = 7*7*16 = 784 
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden])) #shape=(64,)
  
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1)) #shape(64,10)
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels])) #shape(10,)
  
  # Model.
  def model(data, verbose=False):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') #EL STRIDE DE 2 AHORA NO LO APLICA conv2d SINO max_pool 
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    maxpool1 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    if verbose == True:
        print("-> C1.SHAPES WITH STRIDES={}: [input(=data):{} | filter(=layer1_weights):{} -> output(=conv1):{}]".format(
            '[1, 2, 2, 1]', data.shape, layer1_weights.shape, conv1.shape))
        print("-> C1.HIDDEN+MAXPOOL SHAPES: [conv1:{} | layer1_biases:{} -> output(=hidden1):{} -> maxpool1:{}]\n".format(
                conv1.shape, layer1_biases.shape, hidden1.shape, maxpool1.shape))
    
    conv2 = tf.nn.conv2d(maxpool1, layer2_weights, [1, 1, 1, 1], padding='SAME') #EL STRIDE DE 2 AHORA NO LO APLICA conv2d SINO max_pool
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    maxpool2 = tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    if verbose == True:
        print("--> C2.SHAPES WITH STRIDES={}: [input(=maxpool1):{} | filter(=layer2_weights):{} -> output(=conv2):{}]".format(
            '[1, 2, 2, 1]', maxpool1.shape, layer2_weights.shape, conv2.shape))
        print("--> C2.HIDDEN+MAXPOOL SHAPES: [conv2:{} | layer2_biases:{} -> output(=hidden2):{} -> maxpool2:{}]\n".format(
                conv2.shape, layer2_biases.shape, hidden2.shape, maxpool2.shape))

    shape = maxpool2.get_shape().as_list()
    
    reshape2 = tf.reshape(maxpool2, [shape[0], shape[1] * shape[2] * shape[3]])
    prod3 = tf.matmul(reshape2, layer3_weights)
    hidden3 = tf.nn.relu(prod3 + layer3_biases)
    
    if verbose == True:
        print("---> FULLCONN FIRST LAYER SHAPES MATMUL: [hidden reshaped(=reshape2):{} | layer3_weights:{} -> output(=prod3):{}".format(
                reshape2.shape, layer3_weights.shape, prod3.shape))
        print("---> FULLCONN FIRST LAYER HIDDEN: prod3:{} | layer3_biases:{} -> output(=hidden3):{}\n".format(
            prod3.shape, layer3_biases.shape, hidden3.shape))
    
    prod4 = tf.matmul(hidden3, layer4_weights)
    output = prod4 + layer4_biases

    if verbose == True:
        print("----> FULLCONN LAST LAYER SHAPES MATMUL: [hidden3:{} | layer4_weights:{} -> output(=prod4):{}".format(
                hidden3.shape, layer4_weights.shape, prod4.shape))
        print("----> FULLCONN LAST LAYER HIDDEN: prod4:{} | layer4_biases:{} -> output:{}\n".format(
            prod4.shape, layer4_biases.shape, output.shape))    
    
    return output

  # Training computation.
  logits = model(tf_train_dataset, verbose=True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

-> C1.SHAPES WITH STRIDES=[1, 2, 2, 1]: [input(=data):(16, 28, 28, 1) | filter(=layer1_weights):(5, 5, 1, 16) -> output(=conv1):(16, 28, 28, 16)]
-> C1.HIDDEN+MAXPOOL SHAPES: [conv1:(16, 28, 28, 16) | layer1_biases:(16,) -> output(=hidden1):(16, 28, 28, 16) -> maxpool1:(16, 14, 14, 16)]

--> C2.SHAPES WITH STRIDES=[1, 2, 2, 1]: [input(=maxpool1):(16, 14, 14, 16) | filter(=layer2_weights):(5, 5, 16, 16) -> output(=conv2):(16, 14, 14, 16)]
--> C2.HIDDEN+MAXPOOL SHAPES: [conv2:(16, 14, 14, 16) | layer2_biases:(16,) -> output(=hidden2):(16, 14, 14, 16) -> maxpool2:(16, 7, 7, 16)]

---> FULLCONN FIRST LAYER SHAPES MATMUL: [hidden reshaped(=reshape2):(16, 784) | layer3_weights:(784, 64) -> output(=prod3):(16, 64)
---> FULLCONN FIRST LAYER HIDDEN: prod3:(16, 64) | layer3_biases:(64,) -> output(=hidden3):(16, 64)

----> FULLCONN LAST LAYER SHAPES MATMUL: [hidden3:(16, 64) | layer4_weights:(64, 10) -> output(=prod4):(16, 10)
----> FULLCONN LAST LAYER HIDDEN: prod4:(16, 10) | layer4_biases:(10,)

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---